In [48]:
import pandas as pd
import tensorflow as tf

In [56]:
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
from sklearn.model_selection import KFold
from keras.wrappers.scikit_learn import KerasClassifier

In [2]:
df = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/PREDICT_data/4_2_tn_local_descriptor_bayesian.tsv")

In [73]:
indep = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/PREDICT_data/2_2_indep_descriptor.tsv")

In [3]:
print len(df)
df.head(1)

12364


,drug_id,disease_id,association,C0000727,C0000731,C0000737,C0000768,C0000772,C0000786,C0000814,...,387585,388165,55219,55973,65264,80350,84624,84722,91851,406216
0,DB01156,157300,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
print df.columns.values[:1849]
print df.columns.values[:1847]

['drug_id' 'disease_id' 'association' ..., 'C2718128' '100188812'
 '100188813']
['drug_id' 'disease_id' 'association' ..., 'C2363731' 'C2607947' 'C2718128']


In [10]:
asso = df[['association']]
drug = df[df.columns.values[3:1847].tolist()]
disease = df[df.columns.values[1848:].tolist()]

In [18]:
ass = asso.values
dru = drug.values
dise = disease.values

In [74]:
i_ass = indep[['association']].values
i_drug = indep[indep.columns.values[3:1847].tolist()].values
i_disease = indep[indep.columns.values[1848:].tolist()].values

In [22]:
len(dise[0])

3520

In [23]:
import numpy as np
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization, Activation, Merge
from keras.optimizers import Adam
from keras import backend as K

In [39]:
from keras.layers import Input, Dense, merge, Flatten, Convolution2D, MaxPooling2D, Dropout, BatchNormalization, Activation, Embedding, LSTM, Bidirectional
from keras.layers import Convolution1D, GlobalMaxPooling1D

In [45]:
def model(lr=1e-3,dropout=0.7):
    
    model_dr = Sequential()
    model_di = Sequential()
    
    model_dr.add(Dense(output_dim=1500, input_dim=1844, init='glorot_uniform'))
    model_dr.add(BatchNormalization())
    model_dr.add(Activation('relu'))
#     model_dr.add(Dropout(dropout))
    
    model_di.add(Dense(output_dim=3000, input_dim=3520, init='glorot_uniform'))
    model_di.add(BatchNormalization())
    model_di.add(Activation('relu'))
#     model_di.add(Dropout(dropout))
    
    merged = Merge([model_dr, model_di], mode='concat')
    
    model_t = Sequential()
    model_t.add(merged)
    
    model_t.add(Dense(output_dim=1000, input_dim=1500+3000, init='glorot_uniform'))
    model_t.add(BatchNormalization())
    model_t.add(Activation('relu'))
    model_t.add(Dropout(dropout))
    
    model_t.add(Dense(1,activation='sigmoid', init='normal'))
    
    opt = Adam(lr=1e-3)
    model_t.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model_t
    

In [46]:
model_t = model()

In [69]:
K.get_session().run(tf.global_variables_initializer())

In [75]:
K.get_session().run(tf.global_variables_initializer())
for _ in xrange(10):
    history = model_t.fit([dru,dise], ass, nb_epoch=1, batch_size=50, shuffle=True, verbose=1, validation_data=([i_drug,i_disease],i_ass))
    prediction = model_t.predict([i_drug,i_disease])
    AUC = roc_auc_score(i_ass[:,0],prediction)
    precision, recall, thresholds = precision_recall_curve(i_ass[:,0],prediction)
    AUPR = auc(recall,precision)
    print "\tArea Under ROC Curve(AUC): %0.3f" % AUC
    print "\tArea Under PR Curve(AUPR): %0.3f" % AUPR

Train on 12364 samples, validate on 2539 samples
Epoch 1/1
12364/12364 [==============================] - 10s - loss: 0.1689 - acc: 0.9493 - val_loss: 3.0649 - val_acc: 0.5203
	Area Under ROC Curve(AUC): 0.617
	Area Under PR Curve(AUPR): 0.059
Train on 12364 samples, validate on 2539 samples
Epoch 1/1
12364/12364 [==============================] - 9s - loss: 0.0751 - acc: 0.9791 - val_loss: 4.2645 - val_acc: 0.3111
	Area Under ROC Curve(AUC): 0.615
	Area Under PR Curve(AUPR): 0.196
Train on 12364 samples, validate on 2539 samples
Epoch 1/1
12364/12364 [==============================] - 9s - loss: 0.0450 - acc: 0.9882 - val_loss: 4.3602 - val_acc: 0.2418
	Area Under ROC Curve(AUC): 0.632
	Area Under PR Curve(AUPR): 0.171
Train on 12364 samples, validate on 2539 samples
Epoch 1/1
12364/12364 [==============================] - 9s - loss: 0.0434 - acc: 0.9894 - val_loss: 4.7557 - val_acc: 0.2343
	Area Under ROC Curve(AUC): 0.617
	Area Under PR Curve(AUPR): 0.176
Train on 12364 samples, val

In [57]:
from sklearn.model_selection import StratifiedKFold

In [66]:
skf = StratifiedKFold(n_splits=10, shuffle=True) # n_splits = k (k fold라서.), pos:neg의 비율을 고려해서 k 개의 subgroup으로 나누어줌.
for tra, val in skf.split(dru,ass[:,0]):
    dr_tr = dru[tra]
    di_tr = dise[tra]
    y_tr = ass[tra]
    
    dr_val = dru[val]
    di_val = dise[val]
    y_val = ass[val]
    
    break

In [77]:
%sudo cp asol.ipynb /home/share/aschoi/

ERROR: Line magic function `%sudo` not found.
